In [1]:
import random
import cv2
from ultralytics import YOLO
import time

In [2]:
def print_board(board):
    for row in board:
        print(" | ".join(row))
        print("-" * 5)

def check_winner(board, player):
    # Check rows, columns, and diagonals for a win
    for i in range(3):
        if all(board[i][j] == player for j in range(3)) or all(board[j][i] == player for j in range(3)):
            return True
    if all(board[i][i] == player for i in range(3)) or all(board[i][2 - i] == player for i in range(3)):
        return True
    return False

def is_board_full(board):
    return all(board[i][j] != ' ' for i in range(3) for j in range(3))

def get_empty_cells(board):
    return [(i, j) for i in range(3) for j in range(3) if board[i][j] == ' ']

def player_move(board):
    while True:
        try:
            row = int(input("Enter the row (0, 1, or 2): "))
            col = int(input("Enter the column (0, 1, or 2): "))
            if board[row][col] == ' ':
                return row, col
            else:
                print("Cell already occupied. Try again.")
        except (ValueError, IndexError):
            print("Invalid input. Please enter a number between 0 and 2.")

def minimax(board, depth, maximizing_player):
    if check_winner(board, 'O'):
        return 1
    elif check_winner(board, 'X'):
        return -1
    elif is_board_full(board):
        return 0

    if maximizing_player:
        max_eval = float('-inf')
        for i, j in get_empty_cells(board):
            board[i][j] = 'O'
            eval = minimax(board, depth + 1, False)
            board[i][j] = ' '
            max_eval = max(max_eval, eval)
        return max_eval
    else:
        min_eval = float('inf')
        for i, j in get_empty_cells(board):
            board[i][j] = 'X'
            eval = minimax(board, depth + 1, True)
            board[i][j] = ' '
            min_eval = min(min_eval, eval)
        return min_eval

def best_move(board):
    best_val = float('-inf')
    best_move = None
    for i, j in get_empty_cells(board):
        board[i][j] = 'O'
        move_val = minimax(board, 0, False)
        board[i][j] = ' '
        if move_val > best_val:
            best_move = (i, j)
            best_val = move_val
    return best_move



In [3]:
def crop_image(image, coordinates):
    x, y, w, h = coordinates
    cropped_image = image[y:y+h, x:x+w]
    return cropped_image

def get_object_in_region(region_image, model,i,j):
    results = model(region_image, save=False)
    if results[0]:
        print(i)
        print(j)
        print("here")
        print(results[0].names[results[0].boxes.cls.numpy()[0]])
        return results[0].names[results[0].boxes.cls.numpy()[0]]
    return None


In [8]:
def update_board_from_image(board, model,coors):
    # Load the image
#     image = cv2.imread(image_path)
    cap = cv2.VideoCapture(0)

    # Automatically set the initial cropping region for subsequent frames
    crop_x, crop_y, crop_w, crop_h = coors
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        cropped_image = frame[crop_y:crop_y+crop_h, crop_x:crop_x+crop_w]
        cv2.imshow("Cropped Region", cropped_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        height, width, _ = cropped_image.shape
        region_height = height // 3
        region_width = width // 3

        for i in range(3):
            for j in range(3):
                region = cropped_image[i * region_height: (i + 1) * region_height,
                                   j * region_width: (j + 1) * region_width]

            # Get the object in the region using YOLO
                detected_object = get_object_in_region(region, model,i,j)
   
                cv2.imshow("Region",region)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
            # Update the board based on the detected object
                if detected_object == 'teddy bear':
                    board[i][j] = 'X'
                elif detected_object == 'bottle':
                    board[i][j] = 'O'

    # Display the image with rectangles around detected regions
        cv2.imshow("Detected Regions", cropped_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        break
    return board

#     # Ask the user to crop the image
#     coordinates = cv2.selectROI("Select Region", image, False, False)
#     cv2.destroyAllWindows()

#     # Crop the image based on user selection
#     cropped_image = crop_image(image, coordinates)

    # Display the cropped region


    # Define 9 virtual regions in the cropped image for Tic-Tac-Toe grid


In [9]:
def play_tic_tac_toe():
    yolo_model = YOLO('yolov8n.pt')
    board = [[' ' for _ in range(3)] for _ in range(3)]
    player_symbol = 'X'
    computer_symbol = 'O'
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    coordinates = cv2.selectROI("Select Region", frame, False, False)
    cv2.destroyAllWindows()
    initial_cropped_frame = crop_image(frame, coordinates)
    cv2.imshow("Init Crop", initial_cropped_frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cap.release()
    
    print("Welcome to Tic-Tac-Toe!")

    while True:
        print("Start of round board")
        print_board(board)
        # Player move
        time.sleep(5)
        board=update_board_from_image(board,yolo_model,coordinates)
        print("After Player Move")
        print_board(board)
#         player_row, player_col = player_move(board)
#         board[player_row][player_col] = player_symbol

        # Check if the player wins
        if check_winner(board, player_symbol):
            print_board(board)
            print("Congratulations! You win!")
            break

        # Check if the board is full (tie)
        if is_board_full(board):
            print_board(board)
            print("It's a tie!")
            break

        # Computer move
        print("Computer's turn:")
        computer_row, computer_col = best_move(board)
        board[computer_row][computer_col] = computer_symbol
        time.sleep(10)
        # Check if the computer wins
        if check_winner(board, computer_symbol):
            print_board(board)
            print("Computer wins! Better luck next time.")
            break

        # Check if the board is full (tie)
        if is_board_full(board):
            print_board(board)
            print("It's a tie!")
            break




In [12]:
if __name__ == "__main__":
    # Load YOLO model
    
    play_tic_tac_toe()

Welcome to Tic-Tac-Toe!
Start of round board
  |   |  
-----
  |   |  
-----
  |   |  
-----

0: 640x640 (no detections), 129.0ms
Speed: 4.0ms preprocess, 129.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 bottles, 170.6ms
Speed: 5.0ms preprocess, 170.6ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
0
1
here
bottle

0: 640x640 (no detections), 164.3ms
Speed: 9.0ms preprocess, 164.3ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 bottles, 156.6ms
Speed: 8.0ms preprocess, 156.6ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)
1
0
here
bottle

0: 640x640 (no detections), 175.6ms
Speed: 8.0ms preprocess, 175.6ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 187.5ms
Speed: 7.0ms preprocess, 187.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 193.5ms
Speed: 6.0ms preprocess, 193.5ms inference, 4.

KeyboardInterrupt: 